# Guerlédan 2023/10: seabot

In [ ]:
import os
from glob import glob

import xarray as xr
import pandas as pd
import numpy as np
from scipy.signal import lfilter

#%matplotlib inline
import matplotlib.pyplot as plt
import hvplot.pandas # noqa
from cmocean import cm

import pynsitu as pin

In [ ]:
root_path = "/Users/aponte/Current_Projects/ensta/guerledan_202310/data_seabot"

data_files = sorted(glob(os.path.join(root_path, "*.txt")))
data_files

In [ ]:
def load_txt(file):
    columns = [
        "time_posix", 
        "depth_raw", "velocity_raw", 
        "depth", "velocity", "temperature", "altitude",
    ]
    df = pd.DataFrame(np.loadtxt(file).T, columns=columns)
    df["time"] = pd.to_datetime(df["time_posix"], unit="s")
    df = df.set_index("time")
    df = df.resample("1s").mean()
    append_depth_filtered(df, 2.)
    dt = (df.reset_index()["time"].diff().bfill() / pd.Timedelta("1s")).values
    df["velocity_filtered"] = df["depth_filtered"].diff().bfill().values / dt
    return df

def append_depth_filtered(df, tau):
    """ filter pressure, inplace """
    #dt, tau = 1, 2.5
    dt = 1
    alpha = dt/2/tau
    b, a = [alpha], [1, -(1-alpha)]
    df["depth_filtered"] = lfilter(b, a, df.depth.bfill())

# follows Thomas implementation
#a[0]*y[n] = b[0]*x[n] + b[1]*x[n-1] + ... + b[M]*x[n-M]
#                      - a[1]*y[n-1] - ... - a[N]*y[n-N]

# T_(n+1) = T_(n) * (1-alpha) + mesure*(alpha)
# alpha = 0.04


In [ ]:
DF = [load_txt(f) for f in data_files]  


### overview of all dives

In [ ]:
colors = pin.get_cmap_colors(len(DF))

def get_hv_plot(D, v, revert_yaxis=False):
    
    p = None
    for df, c in zip(D, colors):
        if p is None:
            p = df[v].hvplot(color=c, grid=True)
        else:
            p = p * df[v].hvplot(color=c)
    
    if revert_yaxis:
        p = p.opts(invert_yaxis=True)
    
    return p

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,4))
for df, c in zip(DF, colors):
    ax.plot(df.index, -df.depth, color=c)
ax.grid()

In [ ]:
get_hv_plot(DF, "depth", revert_yaxis=True)

In [ ]:
# manually search for core deployment time periods
df = DF[9]
(df["depth"].hvplot(grid=True)+df["velocity"].hvplot(grid=True)).cols(1)

In [ ]:
# keep only isotherm following part
S = [
    slice("2023/10/10 20:36:00","2023/10/11 07:00:00"),
    slice("2023/10/10 20:49:00","2023/10/11 07:00:00"),
    slice("2023/10/10 20:50:00","2023/10/11 07:00:00"),
    slice("2023/10/10 20:45:00","2023/10/11 07:00:00"),
    slice("2023/10/10 20:45:01","2023/10/11 06:33:32"),
    #
    slice("2023/10/11 10:30:00","2023/10/11 16:14:00"),
    slice("2023/10/11 10:25:00","2023/10/11 16:15:00"),
    slice("2023/10/11 10:45:00","2023/10/11 16:15:00"),
    #
    slice("2023/10/12 12:46:00","2023/10/13 07:40:00"),    
    slice("2023/10/12 19:32:00","2023/10/13 07:43:40"),    
    slice("2023/10/12 12:46:40","2023/10/13 07:40:00"),    
    slice("2023/10/12 19:22:00","2023/10/13 09:08:40"),    
]
DT = [df.loc[s] for df, s in zip(DF, S)]

In [ ]:
(get_hv_plot(DT, "depth", revert_yaxis=True) + get_hv_plot(DT, "temperature")).cols(1)

In [ ]:
# isotherm following missions
iso_T = [0, 1, 2, 3, 4, 5, 6, 7, 8, 10]

(get_hv_plot([DT[i] for i in iso_T], "depth", revert_yaxis=True) 
 + get_hv_plot([DT[i] for i in iso_T], "temperature")
).cols(1)

---

## temperature delay

#### 1. simple offset on temperature: 

Seems to work within the water column but not at deepest levels nor at the surface

In [ ]:
df = DF[0]

fig, ax = plt.subplots(1,1)

delays = ["0s", "-2s", "-4s"]
_colors = pin.get_cmap_colors(len(delays))

for delay, c in zip(delays, _colors):
    _df = df.copy()
    _df["temperature"] = _df["temperature"].shift(freq=delay) # avance
    #_df.plot.scatter("temperature","depth", s=1, c=c, ax=ax, label=delay)
    ax.plot(_df.temperature, _df.depth, color=c, label=delay)

ax.invert_yaxis()
ax.grid()
ax.legend()

#### 2. low-pass filter depth instead

seems to behave similarly, 2s seconds seems a reasonable choice

In [ ]:
df = DF[0]

fig, ax = plt.subplots(1,1)

taus = [2, 3, 4]
_colors = pin.get_cmap_colors(len(taus))

for tau, c in zip(taus, _colors):
    _df = df.copy()
    append_depth_filtered(_df, tau)
    ax.plot(_df.temperature, _df.depth_filtered, label=f"tau={tau}")

ax.invert_yaxis()
ax.grid()
ax.legend()

---

## isotherm following missions

In [ ]:
for i in iso_T:
    
    df = DT[i]
    
    #df["depth_anomaly"] = df["depth"] - df["depth"].mean()
    df["depth_anomaly"] = df["depth_filtered"] - df["depth_filtered"].mean()
    df["temperature_anomaly"] = df["temperature"] - df["temperature"].mean()

    dTdz = 1.2 # degC/m
    df["isotherm_displacement"] = df["temperature_anomaly"]/dTdz + df["depth_anomaly"]
    # we compensate for the float vertical displacement
    
    #df["temperature_rate_of_change"] =  (df["temperature"].diff()).bfill()

    # compute estimated
    #df["velocity_water"] = -df["temperature_rate_of_change"]/dTdz - df["velocity"]    
    dt = (df.reset_index()["time"].diff().bfill() / pd.Timedelta("1s")).values
    df["velocity_water"] = df["isotherm_displacement"].diff().bfill().values / dt

### global metrics of regulation

In [ ]:
bins = np.arange(13.8, 14.2, .005)

fig, axes = plt.subplots(5,2, figsize=(7,7), sharex=True, sharey=True)

_i = 0
for i in iso_T:
    ax = axes.flatten()[_i]
    df = DT[i]
    df["temperature"].plot.hist(
        ax=ax, bins=bins, density=True, 
        color=colors[i], 
        label=f"{i} - "+df.index[0].strftime("%Y/%m/%d %H:%M"),
    )
    _i+=1
    ax.grid()
    ax.legend()

In [ ]:
ds_mean = xr.concat([df.mean().to_frame().T.to_xarray() for df in DT], dim="deployment").squeeze()
ds_std = xr.concat([df.std().to_frame().T.to_xarray() for df in DT], dim="deployment").squeeze()

#### temperature std during regulation

not bad ... 😁

We must be close to instrument relative accuracy

In [ ]:
print(ds_std["temperature_anomaly"].isel(deployment=iso_T).values)

The approximate stratification is of about 1.2 degC/m (see diagnostics below).
This enables to diagnose an equivalent vertical displacement:

In [ ]:
print(ds_std["temperature_anomaly"].isel(deployment=iso_T).values /dTdz)

2-3 cm is less that the vertical displacement (see next diagnostics) which confirms that depth variations do indeed reflect isotherm displacements

#### depth std during regulation

This translates isotherm typical displacement amplitude.

The largest amplitude for last two deployments indicates the larger variability in the lake during the last deployment

In [ ]:
print(ds_std["depth_anomaly"].isel(deployment=iso_T).values)

### isotherm displacement

In [ ]:
df = DT[4] 

# show mean depth, temperature for this deployment
df["depth"].mean(), df["temperature"].mean()

In [ ]:
(
    df["depth_anomaly"].hvplot(grid=True) 
    * df["isotherm_displacement"].hvplot() 
    *  (df["temperature_anomaly"]/dTdz).rename("T'/ (dT/dz)").hvplot()
)

The float is able to follow low-frequency (> minutes) fluctuations but not higher temperature fluctuations.
This limit may be related to the regulation response parameters (relaxation feedback constants).

Because of the temperature sensor, we can in theory compensate for this inability to follow isotherm.
This should thus be non-blocking for oceanographic applications

In [ ]:
(
    df["velocity_filtered"].hvplot(grid=True) 
    * df["velocity_water"].hvplot()
)

### compute spectra

In [ ]:
E = df.ts.spectrum(unit="1s", nperseg="1H").to_xarray()

In [ ]:
E = E.where(E.frequency>0, drop=True)

fig, ax = plt.subplots(1,1)

E["depth_anomaly"].plot(ax=ax, label="float displacement")
E["isotherm_displacement"].plot(ax=ax, label="isotherm displacement")

# buoyancy frequency
N = np.sqrt( 9.81 * 2e-4 * dTdz )
print(f"Buoyancy period = {2*np.pi/N/60:.1f} min")
ax.axvline(N/2/np.pi, color="k", ls="--")

ax.set_xscale("log")
ax.set_yscale("log")

ax.legend()

---

## thermocline scanning experiment

In [ ]:
def plot_scatter(df):

    fig, ax = plt.subplots(1,1, figsize=(10, 5))

    _df = df.reset_index()
    _df = _df.loc[_df.velocity>0] # downward profiles

    _df.plot.scatter("time", "depth_filtered", c="temperature", s=8, ax=ax, cmap=cm.thermal)
    ax.grid()

    ax.invert_yaxis()

#

In [ ]:
plot_scatter(DT[9])
plot_scatter(DT[11])

In [ ]:
# with hvplot

#df = DT[9]
#df = df.reset_index()
#df = df.loc[df.velocity>0] # select downward profiles

#_df.hvplot.scatter("time", "depth", c="temperature", s=10, ax=ax, cmap=cm.thermal).opts(invert_yaxis=True)

#### depth-time bin average

In [ ]:
def depth_time_bin_average(df, dz=.1, freq="10T", ascent=False):

    df = df.reset_index()
    if ascent is None:
        pass
    elif ascent:
        df = df.loc[df.velocity<0] # select downward profiles
    else:
        df = df.loc[df.velocity>0] # select downward profiles
        

    dbins = np.arange(15, 20.5, dz)
    tbins = pd.date_range(df.time.iloc[0], df.time.iloc[-1], freq=freq)

    #df["depth_cut"] = pd.cut(df["depth"], dbins)
    df["depth_cut"] = pd.cut(df["depth_filtered"], dbins)
    df["time_cut"] = pd.cut(df["time"], tbins)


    dfb = df.groupby(["depth_cut", "time_cut"]).mean().reset_index()

    dfb["depth"] = dfb["depth_cut"].apply(lambda i: i.left)
    dfb["time"] = dfb["time_cut"].apply(lambda i: i.left)

    ds = dfb.set_index(["depth", "time",]).to_xarray()

    return ds


def bin_plot(df, **kwargs):

    ds = depth_time_bin_average(df, **kwargs)

    fig, ax = plt.subplots(1,1)

    #DT[8]["depth"].plot(ax=ax, color="k")
    #DT[10]["depth"].plot(ax=ax, color="0.3")
    DT[8]["depth_filtered"].plot(ax=ax, color="k")
    DT[10]["depth_filtered"].plot(ax=ax, color="0.3")

    ds["temperature"].plot(ax=ax, cmap=cm.thermal)
    ds["temperature"].plot.contour(ax=ax, levels=[12, 14, 16], colors="w")

    ax.invert_yaxis()
    ax.grid()

In [ ]:
bin_plot(DT[9], ascent=None)
bin_plot(DT[9], ascent=True)
bin_plot(DT[9], ascent=False)

In [ ]:
bin_plot(DT[11])

### archived material ...

In [ ]:
df = DT[9]

_df = df.reset_index()
_df["mask"] = 0.
_df["mask"].where(_df.velocity>0, other=1., inplace=True)

_df["mask_cumsum"] = _df["mask"].cumsum()

_df["mask_cumsum"].hvplot()

In [ ]:
df = DT[9]

_df = df.reset_index()

_df = _df.loc[_df.velocity>0] # downward profiles

dt = _df["time"].diff()/pd.Timedelta("1T")
print("Time intervals in minutes:")
print(np.unique(dt))

#(_df["time"].diff()/pd.Timedelta("1s")).plot.hist(bins=np.arange(0,10,1), log=True) #.plot()

In [ ]:
_df = df.reset_index()

_df = _df.loc[_df.velocity>0] # downward profiles
_dt = _df["time"].diff()/pd.Timedelta("1T")
bottom = _df.loc[_dt>1]

_df = df.reset_index()
_df = _df.loc[_df.velocity<0] # downward profiles
_dt = _df["time"].diff()/pd.Timedelta("1T")
top = _df.loc[_dt>1]

In [ ]:
(df.reset_index()["depth"].hvplot()
 * bottom["depth"].hvplot() 
 * top["depth"].hvplot() 
)

---
## compute vertical stratification

In [ ]:
df = DF[0]

In [ ]:
dz = .25
depth_bins = np.arange(0,df["depth"].max(), dz)

In [ ]:
df["depth_cut"] = pd.cut(df.depth, depth_bins)

In [ ]:
df_bin = df.groupby(df.depth_cut).mean().set_index("depth")

In [ ]:
fig, ax = plt.subplots(1,1)
ax.scatter(df.temperature, -df.depth, s=.1)
ax.plot(df_bin["temperature"], -df_bin.index, color="k", lw=2)
ax.grid()

In [ ]:
(17-14)/(17.5-15)

In [ ]:
(df_bin["temperature"].diff()/dz).plot()